In [3]:
import logging
import pandas as pd
import numpy as np
from numpy import random
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import re

%matplotlib inline

import itertools
import os
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix,classification_report
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Dropout, TimeDistributed, Embedding
from keras.preprocessing import text, sequence
from keras import utils

from keras import backend as K
K.tensorflow_backend._get_available_gpus()


[]

In [4]:
df_train = pd.read_csv('input/train.csv')
# df = df[pd.notnull(df['tags'])]
#df_train.head(10)

In [5]:
train_posts, test_posts, train_tags, test_tags = train_test_split(df_train['title'], 
                                                                  df_train['Category'], 
                                                                  test_size=0.3, 
                                                                  random_state=40)

In [6]:
df_train_subset = pd.read_csv('input/subset_training.csv')

In [44]:
a = np.array([[1,2,3],[4,5,6]]) 
print(a)
print(a.shape)
a = np.append(a, a, axis=1);
print(a)
print(a.shape)

[[1 2 3]
 [4 5 6]]
(2, 3)
[[1 2 3 1 2 3]
 [4 5 6 4 5 6]]
(2, 6)


In [ ]:
max_words = 50
tokenize = text.Tokenizer(num_words=max_words, char_level=False)

is_class0 = df_train['Category']==0
class_0 = df_train[is_class0]


tokenize.fit_on_texts(class_0['title']) # only fit on subset data
x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)
 
for i in range(1,58):
    print (i)
    boolean_filter = df_train['Category']==i
    class_i = df_train[boolean_filter]
    
    max_words = 50
    tokenize = text.Tokenizer(num_words=max_words, char_level=False)
    
    tokenize.fit_on_texts(class_i['title'])
    x_train = np.append(x_train, tokenize.texts_to_matrix(train_posts), axis=1);
    x_test = np.append(x_test, tokenize.texts_to_matrix(test_posts), axis=1);

1
2


In [9]:
encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

In [10]:
num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [11]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (466630, 2000)
x_test shape: (199985, 2000)
y_train shape: (466630, 58)
y_test shape: (199985, 58)


In [12]:
batch_size = 256
epochs = 5

In [13]:
x_train[1,:]

array([0., 1., 1., ..., 0., 0., 0.])

In [14]:
train_posts[1]

'etude house precious mineral any cushion pearl aura puff'

In [15]:
# Build the model
model = Sequential()
model.add(LSTM(1000,input_shape=(1,max_words)))
model.add(Dropout(0.6))
model.add(Dense(512, activation='elu'))
model.add(Dropout(0.6))

#model.add(Dense(num_classes, activation='elu'))
#model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='elu'))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [16]:
history = model.fit(x_train.reshape((x_train.shape[0],1,x_train.shape[1])), y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 419967 samples, validate on 46663 samples
Epoch 1/5
126208/419967 [========>.....................] - ETA: 20:30 - loss: 1.3482 - acc: 0.6315

KeyboardInterrupt: 

In [ ]:
score = model.evaluate(x_test.reshape((x_test.shape[0],1,x_test.shape[1])), y_test,
                       batch_size=batch_size, verbose=1)
print('Test accuracy:', score[1])

In [ ]:

y_predicted = model.predict_classes(x_test.reshape((x_test.shape[0],1,x_test.shape[1])), verbose=1)

In [ ]:
y_test_num = np.argmax(y_test, axis=1)

In [ ]:
print(classification_report(y_test_num, y_predicted))

In [ ]:
df_public = pd.read_csv('input/test.csv')
df_public.head(10)

In [ ]:
x_public = tokenize.texts_to_matrix(df_public["title"])

In [ ]:
preds = model.predict(x_public.reshape((x_public.shape[0],1,x_public.shape[1])), verbose=1)

In [ ]:
df_public['Category'] = [np.argmax(pred) for pred in preds]
df_submit = df_public[['itemid', 'Category']].copy()
df_submit.to_csv('submission.csv', index=False)

In [ ]:
model.save('ThurMar21.h5')